In [1]:
# module
import numpy as np
import pandas as pd
import tqdm
from haversine import haversine
from tqdm import tqdm_pandas, tqdm

In [2]:
#load data
cw = pd.read_csv('횡단보도_완료.csv', encoding = 'cp949')
fluid = pd.read_csv('강서구_지하철_유동인구.csv', encoding = 'cp949')

In [3]:
fluid

,역명,위도,경도,승차총승객수,하차총승객수
0,방화,37.577669,126.812822,6402.487179,6107.538462
1,개화산,37.572458,126.806838,5385.230769,5159.564103
2,송정,37.561411,126.812052,6145.179487,5809.076923
3,마곡,37.562182,126.826930,9872.205128,9996.282051
4,발산,37.562182,126.826930,19879.820513,20772.974359
5,우장산,37.548864,126.836330,13611.076923,13370.076923
6,화곡,37.541585,126.840436,27246.282051,25422.128205
7,까치산,37.531810,126.846706,27313.358974,26170.256410
8,개화,37.578608,126.798153,2470.230769,1631.820513
9,김포공항,37.562434,126.801058,25718.538462,24021.153846


In [4]:
# 차 사고
cw.columns = ['crosswalk_id', 'cw_lat', 'cw_lon']

# 횡단보도
fluid.columns = ['tube', 'sub_lon', 'sub_lat', 'on', 'off']

In [5]:
# key 만들기
cw['key'] = 0
fluid['key'] = 0

In [6]:
# 데이터 합치기
prod = pd.merge(cw, fluid, on = 'key', how = 'outer')

In [7]:
## 하버사인 거리
lat1 = np.deg2rad(prod['cw_lat'])
lat2 = np.deg2rad(prod['sub_lat'])
lon1 = np.deg2rad(prod['cw_lon'])
lon2 = np.deg2rad(prod['sub_lon'])
dlat = lat2 - lat1 
dlon = lon2 - lon1 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [8]:
# 미터거리 구하기
prod['distance'] = c * 6371 * 1000

In [9]:
# 상세한 확인
prod['distance'].describe()

count    43956.000000
mean      2965.212759
std       1823.967437
min          3.101408
25%       1503.132426
50%       2647.083469
75%       4176.811197
max       9167.508779
Name: distance, dtype: float64

In [10]:
# 거리가 가까운 순서대로 정렬
p = prod.sort_values(by=['distance'])

In [11]:
# 가장 가까운 값들로 저장
pp = p.drop_duplicates(subset = 'crosswalk_id')[['crosswalk_id', 'tube', 'on', 'off', 'distance']]

In [12]:
pp.to_csv('지하철유동인구_거리포함.csv', index = False, encoding = 'cp949')

In [13]:
pp['distance'].describe()

count    2442.000000
mean      456.335783
std       260.634602
min         3.101408
25%       266.709708
50%       421.390078
75%       620.739428
max      1513.807853
Name: distance, dtype: float64